# Abstract

Heart disease is a leading cause of death worldwide, and early detection and prevention can save lives. In this project, we analysed data from a 2020 annual CDC survey data of 400k adults related to their health status, and applied different machine learning algorithms aiming to predict chance of one individual having heart disease. The project first performed data cleaning and feature selection to prepare the data for modeling. Then, several machine learning models such as LDA, Random Forest, SVM, Decision Tree, XGBoost, and Neural Network were applied to find the model with the highest sensitivity for the final prediction. The results showed that the XGBoost model performed the best in terms of sensitivity, and thus was selected as the final model for the prediction of heart disease. We created a user interface and linked with our XGBoost model, which allow user to input data about their personal health status and returns whether the patient require further diagnoise or not. This project highlights the importance of data preparation and model selection in achieving accurate results in the field of medical diagnosis.

# Encoding Variables

In [1]:
import numpy as np
import pandas as pd

## Basic information about the dataset

In [2]:
data = pd.read_csv("heart_2020_cleaned.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [4]:
data.shape

(319795, 18)

As the data is cleaned, there is no necessary to check for NA values.

## Interpretation of variables
**Number of unique values for each variable**

In [5]:
data.nunique()

HeartDisease           2
BMI                 3604
Smoking                2
AlcoholDrinking        2
Stroke                 2
PhysicalHealth        31
MentalHealth          31
DiffWalking            2
Sex                    2
AgeCategory           13
Race                   6
Diabetic               4
PhysicalActivity       2
GenHealth              5
SleepTime             24
Asthma                 2
KidneyDisease          2
SkinCancer             2
dtype: int64

In [6]:
for feature in data.columns:
    print(feature, data[feature].unique(),"\n")

HeartDisease ['No' 'Yes'] 

BMI [16.6  20.34 26.58 ... 62.42 51.46 46.56] 

Smoking ['Yes' 'No'] 

AlcoholDrinking ['No' 'Yes'] 

Stroke ['No' 'Yes'] 

PhysicalHealth [ 3.  0. 20. 28.  6. 15.  5. 30.  7.  1.  2. 21.  4. 10. 14. 18.  8. 25.
 16. 29. 27. 17. 24. 12. 23. 26. 22. 19.  9. 13. 11.] 

MentalHealth [30.  0.  2.  5. 15.  8.  4.  3. 10. 14. 20.  1.  7. 24.  9. 28. 16. 12.
  6. 25. 17. 18. 21. 29. 22. 13. 23. 27. 26. 11. 19.] 

DiffWalking ['No' 'Yes'] 

Sex ['Female' 'Male'] 

AgeCategory ['55-59' '80 or older' '65-69' '75-79' '40-44' '70-74' '60-64' '50-54'
 '45-49' '18-24' '35-39' '30-34' '25-29'] 

Race ['White' 'Black' 'Asian' 'American Indian/Alaskan Native' 'Other'
 'Hispanic'] 

Diabetic ['Yes' 'No' 'No, borderline diabetes' 'Yes (during pregnancy)'] 

PhysicalActivity ['Yes' 'No'] 

GenHealth ['Very good' 'Fair' 'Good' 'Poor' 'Excellent'] 

SleepTime [ 5.  7.  8.  6. 12.  4.  9. 10. 15.  3.  2.  1. 16. 18. 14. 20. 11. 13.
 17. 24. 19. 21. 22. 23.] 

Asthma ['Yes' 'No'] 


Body Mass Index (BMI) values is numeric continuous data.

The following predictors are binary: "HeartDisease", "Smoking", "AlcoholDrinking", "Stroke", "DiffWalking", "PhysicalActivity", "Asthma", "KidneyDisease", and "SkinCancer". There are only 2 unique values for each, which can be factored into 1 and 0.

The physical health predictor is a numeric continuous variable providing information about the amount of days of injuries for individual participants over the past 30 days. The larger the value, the poorer physical condition of participant.

Similar to mental health predictor, which indicates the number of days in the past 30 days that the participant does NOT feel well. The larger value indicating a worse mental health status.

## Encoding all variables

**Convert binary to 1/0**

In [7]:
data_binary = data[data.columns].replace({'Yes':1, 'No':0, 'Male':1,'Female':0,'No, borderline diabetes':'0','Yes (during pregnancy)':'1' })
data_binary['Diabetic'] = data_binary['Diabetic'].astype(int)

In [8]:
data_binary.head(5)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,0,55-59,White,1,1,Very good,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,0,80 or older,White,0,1,Very good,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,1,65-69,White,1,1,Fair,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,0,75-79,White,0,0,Good,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,0,40-44,White,0,1,Very good,8.0,0,0,0


**Convert categorial variables to dummy indicator**

In [9]:
categorical_columns = [name for name in data_binary.columns 
                       if data_binary[name].dtype=='O']
categorical_columns

['AgeCategory', 'Race', 'GenHealth']

In [10]:
data_dummy = pd.get_dummies(data=data_binary, columns=categorical_columns, drop_first=False)
data_dummy.head(5)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Diabetic,...,Race_Asian,Race_Black,Race_Hispanic,Race_Other,Race_White,GenHealth_Excellent,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good
0,0,16.60,1,0,0,3.0,30.0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
1,0,20.34,0,0,1,0.0,0.0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,0,26.58,1,0,0,20.0,30.0,0,1,1,...,0,0,0,0,1,0,1,0,0,0
3,0,24.21,0,0,0,0.0,0.0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,23.71,0,0,0,28.0,0.0,1,0,0,...,0,0,0,0,1,0,0,0,0,1


Converting categorical variables into dummy indicators will be later utilized in model prediction.

# Data Cleaning

## Split dataset into train and test data

In [11]:
from sklearn.model_selection import train_test_split

# assume df is the DataFrame and y is the target variable
# split the data into training and test sets with 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(data_dummy.drop(columns=[data_dummy.columns[0]]), 
                                                    data_dummy.iloc[:,0], 
                                                    test_size=0.2, 
                                                    random_state=42)

train_data = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis=1)
test_data = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test)], axis=1)

# display the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("train_data shape:", train_data.shape)
print("test_data shape:", test_data.shape)

X_train shape: (255836, 38)
y_train shape: (255836,)
X_test shape: (63959, 38)
y_test shape: (63959,)
train_data shape: (255836, 39)
test_data shape: (63959, 39)


## Analysis of response variable - HeartDisease

In [12]:
pd.Series.value_counts(train_data.HeartDisease)

0    234055
1     21781
Name: HeartDisease, dtype: int64

The dataset is heavily unbalanced with 292422 respondents without heart disease, and only 27373 of participants shown of having heart disease. Having imbalanced dataset might result in a poor performance of the prediction, as the result might be biased toward the majority class indicaiting no heart disease. For our model, we tend to be biased toward "having heart disease", so in reality a higher chance to be predicted to have "heart disease" to prevent misdiagnose.

## SMOTE Technique
Address the issue we have for imbalanced dataset by synthesizing new examples from the minority class. It involves randomly selecting a minority class example and finding its k nearest neighbors in the feature space. New examples are then generated by interpolating between the selected example and its neighbors. This aim to increase the number of examples in minority class.

SMOTE technique is perfect approach to deal with imbalance problems for binary variable. Hence can be eperfectly utilized for our problem.

In [13]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.datasets import make_classification

X = X_train
y = y_train

print('Before SMOTE:', X.shape, y.shape)


# # create SMOTE object with desired sampling strategy
# sm = SMOTE(sampling_strategy='auto', random_state=42)

# # apply SMOTE to generate new samples
# X_resampled, y_resampled = sm.fit_resample(X, y)

smote = BorderlineSMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X, y)

# after applying SMOTE
print('After SMOTE:', X_resampled.shape, y_resampled.shape)
train_resampled = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)

pd.Series.value_counts(y_resampled)

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (/Users/yutongwu/anaconda3/lib/python3.11/site-packages/sklearn/utils/_param_validation.py)

After SMOTE technique, the number of participants having heart disease is increased to be equal to the number of non-heart disease records. The SMOTE method balance out the data set so that the number of participants with heart disease equals the number of participants that doesn't, so the accuracy of our prediction will largely increase.

## Convert the data after SMOTE into CSV files

X_resampled.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_resampled.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
train_resampled.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

## Scale the data

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_resampled)
X_train_scale = pd.DataFrame(X_train_scale, columns=X_resampled.columns)
X_test_scale = scaler.transform(X_test)
X_test_scale = pd.DataFrame(X_test_scale, columns=X_resampled.columns)
train_data_scale = pd.concat([pd.DataFrame(y_resampled), pd.DataFrame(X_train_scale)], axis=1)
test_data_scale = pd.concat([pd.DataFrame(y_test), pd.DataFrame(X_test_scale)])

## Convert the scaled data into csv

X_train_scale.to_csv('X_train_scale.csv', index=False)
X_test_scale.to_csv('X_test_scale.csv', index=False)
y_resampled.to_csv('y_train_scale.csv', index=False)
y_test.to_csv('y_test_scale.csv', index=False)

# Feature Selection

## Correlation Heatmap

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(data_binary.corr(), cmap='coolwarm', center=0, annot=True)
ax.set_title('Correlation Heatmap', fontsize=20)
ax.tick_params(labelsize=12)
plt.xticks(rotation=45)
plt.show()

## Principal Component Analysis (PCA)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_pca = pca.fit_transform(train_data_scale)

explained_variance_ratio = pca.explained_variance_ratio_

PC1 = pca.fit_transform(train_data_scale)[:,0]
PC2 = pca.fit_transform(train_data_scale)[:,1]
ldngs = pca.components_

scalePC1 = 1.0/(PC1.max() - PC1.min())
scalePC2 = 1.0/(PC2.max() - PC2.min())
features = train_data_scale.columns


In [ ]:
fig, ax = plt.subplots(figsize=(14, 9))

for i, feature in enumerate(features):
    ax.arrow(0, 0, ldngs[0, i], 
             ldngs[1, i], 
             head_width=0.03, 
             head_length=0.03)
    ax.text(ldngs[0, i] * 1.15, 
            ldngs[1, i] * 1.15, 
            feature, fontsize = 18)

ax.set_xlabel('PC1', fontsize=20)
ax.set_ylabel('PC2', fontsize=20)
ax.set_title('Figure 1', fontsize=20)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 9))

scatter = ax.scatter(PC1 * scalePC1, 
                     PC2 * scalePC2, 
                     c=y_resampled, 
                     cmap='viridis')

ax.set_xlabel('PC1', fontsize=20)
ax.set_ylabel('PC2', fontsize=20)
ax.set_title('Figure 1', fontsize=20)

legend1 = ax.legend(*scatter.legend_elements(),
                    loc="lower left", 
                    title="Groups")
ax.add_artist(legend1)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 9))

for i, feature in enumerate(features):
    ax.arrow(0, 0, ldngs[0, i], 
             ldngs[1, i], 
             head_width=0.03, 
             head_length=0.03)
    ax.text(ldngs[0, i] * 1.15, 
            ldngs[1, i] * 1.15, 
            feature, fontsize = 18)

scatter = ax.scatter(PC1 * scalePC1, 
                     PC2 * scalePC2, 
                     c=y_resampled, 
                     cmap='viridis')

ax.set_xlabel('PC1', fontsize=20)
ax.set_ylabel('PC2', fontsize=20)
ax.set_title('Figure 1', fontsize=20)

legend1 = ax.legend(*scatter.legend_elements(),
                    loc="lower left", 
                    title="Groups")
ax.add_artist(legend1)

In [ ]:
plt.figure(figsize=(14,9))
 
for i, feature in enumerate(features):
    plt.arrow(0, 0, ldngs[0, i], 
             ldngs[1, i], 
              head_width=0.03, 
             head_length=0.03)
    plt.text(ldngs[0, i] * 1.15, 
            ldngs[1, i] * 1.15, 
            feature, fontsize=18)
 
sns.scatterplot(x=PC1 * scalePC1,
                y=PC2 * scalePC2, 
                hue=y_resampled,
                palette="viridis")
 
plt.xlabel('PC1', fontsize=20)
plt.ylabel('PC2', fontsize=20)
plt.title('Figure 2', fontsize=20)

In [ ]:
import matplotlib.pyplot as plt
pca = PCA()
pca.fit(X_train)

plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, 'bo-', linewidth=2)
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained')
plt.show()

In [ ]:
# Get the loadings for each variable on each principal component
loadings = pca.components_.T[:, :5]

# Create a dataframe to store the loadings
pc_table = pd.DataFrame(loadings, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'], index=X.columns)

# Display the PC table
print(pc_table)

## L1 regularization

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from statistics import mean

In [ ]:
lasso_reg = Lasso(alpha=0.1)

# Fit the model to the training data
lasso_reg.fit(X_train, y_train)

# Make predictions on the test data
y_pred = lasso_reg.predict(X_test)

# Compute the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Print the coefficients of the model
print("Coefficients:", lasso_reg.coef_)

In [ ]:
alphas = np.logspace(-4, 1, 50)

# Initialize a list to store the coefficients for each alpha value
coefs = []

# Fit the model for each alpha value and store the coefficients
for a in alphas:
    lasso_reg = Lasso(alpha=a)
    lasso_reg.fit(X_train, y_train)
    coefs.append(lasso_reg.coef_)
    
# Get the names or labels of the features
feature_names = X_train.columns

# Create a DataFrame to store the coefficients
df = pd.DataFrame(coefs, columns=feature_names)

# Transpose the DataFrame to have the alpha values as the index
df = df.transpose()

# Print the resulting table
print(df)

# Plot the coefficients as a function of alpha
plt.figure(figsize=(10, 6))
ax = plt.gca()

# Plot the coefficients for each feature
for i, c in enumerate(coefs[0]):
    ax.plot(alphas, [coef[i] for coef in coefs], label=feature_names[i])

ax.set_xscale('log')
ax.set_xlabel('alpha')
ax.set_ylabel('coefficients')
ax.set_title('L1 Regularization Path')

# Add a legend to the plot
plt.legend(loc='best')

plt.axis('tight')
plt.show()

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.svm import SVC
# from scipy.stats import uniform

# # assuming X_train and y_train are your training data
# # assuming param_distributions is a dictionary of hyperparameters to search over
# # assuming scoring_metric is the metric to use for evaluation (e.g. accuracy, f1-score)
# # assuming cv is the number of cross-validation folds to use
# model = SVC() 
# param_distributions = {'C': np.logspace(-3, 3, 7),
#                        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#                        'gamma': np.logspace(-3, 3, 7)}
# scoring_metric = 'accuracy'
# cv = 5

# random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, 
#                                    scoring=scoring_metric, cv=cv, n_iter=10, random_state=42)
# random_search.fit(X_train, y_train)

# print('Best hyperparameters:', random_search.best_params_)
# print('Best', scoring_metric, 'score:', random_search.best_score_)

# GridSearchCV

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100],
              'penalty': ['l1', 'l2']}

logreg = LogisticRegression()
grid_search = GridSearchCV(logreg, param_grid, cv=5)
grid_search.fit(X_train_scale, y_resampled)
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
y_pred = grid_search.best_estimator_.predict(X_test)


In [ ]:
model = LogisticRegression()
model.fit(X_train_scale, y_resampled)
y_pred = model.predict(X_test_scale)
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

## Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50)],
              'alpha': [0.0001, 0.001, 0.01, 0.1],
              'activation': ['logistic']}
mlp = MLPClassifier(max_iter=1000)
grid_search = GridSearchCV(mlp, param_grid, cv=5)
grid_search.fit(X_train_scale, y_resampled)
print("Best hyperparameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
y_pred = grid_search.best_estimator_.predict(X_test)